In [1]:
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from load_data.prepared_custom_ds import CustomDataset
from utilities.config_load import load_config

import time
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

In [2]:
CONFIG_PATH = "configs/"
config = load_config(CONFIG_PATH, "config.yaml")

In [3]:
config

{'dataset': {'train_img_path': 'dataset/train_v2',
  'test_img_path': 'dataset/test_v2',
  'mask_path': 'dataset/train_masks',
  'reshaped_img_path': 'dataset/reshaped_img',
  'dir_path': 'dataset'},
 'original_img_size': 768,
 'new_img_size': 256,
 'project_path': 'C:/Users/da4nik/Segmentation',
 'model': None}

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def ConvBlock(first_chanels, second_chanels, kernel_size, dropout_rate):
    return nn.Sequential(
        nn.BatchNorm2d(first_chanels),
        nn.Conv2d(first_chanels, second_chanels, kernel_size, padding='same'),
        nn.ReLU(inplace=True),
        nn.Dropout(dropout_rate),
        nn.BatchNorm2d(second_chanels),
        nn.Conv2d(second_chanels, second_chanels, kernel_size, padding='same'),
        nn.ReLU(inplace=True)
    )

In [ ]:
class Unet_Encoder(nn.Module):
    def __init__(self, kernel_size, dropout_rate, nkernels):
        super(Unet_Encoder, self).__init___()
        self.kernel_size = kernel_size
        self.dropuut_rate = dropuut_rate
        self.nkernels = nkernels
        self.conv1 = ConvBlock(3, nkernels, self.kernel_size, self.dropout_rate)
        self.conv2 = ConvBlock(nkernels, nkernels*2, self.kernel_size, self.dropout_rate)
        self.conv3 = ConvBlock(nkernels*2, nkernels*4, self.kernel_size, self.dropout_rate)
        self.conv4 = ConvBlock(nkernels*4, nkernels*8, self.kernel_size, self.dropout_rate)
        self.maxpool_list = nn.ModuleList([nn.MaxPool2d(kernel_size=2) for _ in range(4)])
        self.conv_list = nn.ModuleList([self.conv1, self.conv2, self.conv3, self.conv4])

    def init_weights(self):
        for module in self.modules():
            if isinstance(module, (nn.Linear, nn.Conv2d)):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0.01)

    def forward(self, input):
        list_skips = list()
        for i in range(4):
            skip = self.self.conv_list[i](input)
            input = self.maxpool_list[i](skip1)
            list_skips.append(skip)
        return input, list_skips

In [ ]:
class Unet_Decoder(nn.Module):
    def __init__(self, kernel_size, dropout_rate, nkernels):
        super(Unet_Decoder, self).__init___()
        self.kernel_size = kernel_size
        self.dropout_rate = dropuut_rate
        self.nkernels = nkernels
        self.conv5 = ConvBlock(nkernels*8, nkernels*16, self.kernel_size, self.dropout_rate)
        self.conv6 = ConvBlock(nkernels*16, nkernels*8, self.kernel_size, self.dropout_rate)
        self.conv7 = ConvBlock(nkernels*8, nkernels*4, self.kernel_size, self.dropout_rate)
        self.conv8 = ConvBlock(nkernels*4, nkernels*2, self.kernel_size, self.dropout_rate)
        self.conv_list = nn.ModuleList([self.conv5, self.conv6, self.conv7, self.conv8])
        self.convt_list = nn.ModuleList([nn.ConvTranspose2d(nkernels*(2**(4-i)), nkernels*((2**(4-i))//2), kernel_size=(2, 2), stride=(2, 2)) 
                                           for i in range(4)])

    
    def init_weights(self):
        for module in self.modules():
            if isinstance(module, (nn.Linear, nn.Conv2d, nn.ConvTranspose2d)):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0.01)

    
    def forward(self, input, list_skips):
        for i in range(4):
            if i==0:
                out = self.conv_list[i](input)
                out = self.convt_list[i](out)
            else:
                out = self.conv_list[i](torch.cat((out, list_skips[4-i]), 0))
                out = self.convt_list[i](out)
        return out

In [ ]:
class Model_Unet(nn.Module):
    def __init__(self, kernel_size, dropout_rate, nkernels, output_chanels):
        super(Model_Unet, self).__init___()
        self.output_chanels = output_chanels
        self.kernel_size = kernel_size
        self.dropout_rate = dropuut_rate
        self.nkernels = nkernels
        self.enc_layer = Unet_Encoder(self.kernel_size, self.dropout_rate, self.nkernels)
        self.dec_layer = Unet_Decoder(self.kernel_size, self.dropout_rate, self.nkernels)
        self.conv9 = ConvBlock(self.nkernels*2, self.nkernels, self.kernel_size, self.dropout_rate)
        self.conv10 = nn.Conv2d(self.nkernels, self.output_chanels, (1, 1), padding='same')
        self.activation = nn.Sigmoid()
        

    def init_weights(self):
        for module in self.modules():
            if isinstance(module, (nn.Linear, nn.Conv2d)):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0.01)

    
    def forward(self, input):
        out, list_skips = self.enc_layer(input)
        out = self.dec_layer(out, list_skips)
        out = self.conv9[i](torch.cat((out, list_skips[0]), 0))
        out = self.activation(out)
        return out

In [ ]:
dataset = CustomDataset(X_train, y_train)
vdataset = CustomDataset(X_test, y_test)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=12, num_workers=0, shuffle=True) # + num thread num_workers=6,
vdataloader = torch.utils.data.DataLoader(vdataset, batch_size=12, num_workers=0, shuffle=True)

In [ ]:
def train_step(model, loss_fn, opt, loader):
    loss_per_batches = 0
    elapsed = 0
    start_epoch2 = time.time()
    for i, data in enumerate(loader):

        start_epoch = time.time()
        features, labels = data
        features, labels = features.to(device), labels.to(device)
        opt.zero_grad()
        
        y_pred = model(features)
        loss = loss_fn(y_pred, labels)
        loss.backward()
        
        opt.step()
        
        loss_per_batches += loss
        
        end_epoch = time.time()
        elapsed += (end_epoch - start_epoch)
        
    print("train = " + str(elapsed))
    print("train + load = " + str(time.time() - start_epoch2))
    return loss_per_batches/(i+1)

In [ ]:
def train(model, loss_fn, opt, train_loader, val_loader, save_treshold=10, epochs=50, model_name='model_name'):
        
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = SummaryWriter('runs/' + model_name + '_{}'.format(timestamp))
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', patience=3, verbose=True)
    
    for epoch in range(epochs):
        start_epoch = time.time()
        print('EPOCH {}:'.format(epoch + 1))
        
        model.train()
        avg_loss = train_step(model, loss_fn, opt, train_loader)
        model.eval()
        

In [ ]:
model = Model_Unet(kernel_size=3, dropout_rate=0.15, nkernels=2, output_chanels=1)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9)
model.to(device)

In [3]:
loss_fun = torch.nn.BCELoss()

In [6]:
res = loss_fun(torch.zeros(1, 200, 200), torch.ones(1, 200, 200))

In [7]:
res

tensor(100.)

In [11]:
for i in range(4):
    print(str(2**(4-i)) + " => " + str((2**(4-i))//2))

16 => 8
8 => 4
4 => 2
2 => 1


In [2]:
max = nn.MaxPool2d(2)

In [3]:
ct = nn.ConvTranspose2d(16, 16//2, kernel_size=(2, 2), stride=(2, 2))

In [4]:
res = max(torch.zeros(16, 200, 200))

In [5]:
res.shape

torch.Size([16, 100, 100])

In [6]:
test = ct(torch.zeros(16, 200, 200))

In [7]:
test.shape

torch.Size([8, 400, 400])